In [15]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

In [6]:
url = "https://quotes.toscrape.com/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
data = defaultdict(lambda: defaultdict(list))

In [ ]:
quotes = soup.find_all("div", class_="quote")

for quote in quotes:
    text = quote.find("span", class_="text").get_text(strip=True)
    author = quote.find("small", class_="author").get_text(strip=True)
    tags = [tag.get_text(strip=True) for tag in quote.find_all("a", class_="tag")]

    for tag in tags:
        data[author][tag].append(text)

pprint(data)

In [24]:
next_button = soup.find("li", class_="next")
next_href = next_button.find("a")
next_href["href"]

'/page/2/'

In [26]:
url = "https://quotes.toscrape.com"
data = defaultdict(lambda: defaultdict(list))

page_count = 0

while url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    page_count += 1
    print(page_count)

    for quote in quotes:
        text = quote.find("span", class_="text").get_text(strip=True)
        author = quote.find("small", class_="author").get_text(strip=True)
        tags = [tag.get_text(strip=True) for tag in quote.find_all("a", class_="tag")]

        for tag in tags:
            data[author][tag].append(text)
    
    next_button = soup.find("li", class_="next")

    if next_button:
        next_href = next_button.find("a")["href"]
        url = "https://quotes.toscrape.com/" + next_href
    else:
        url = None

1
2
3
4
5
6
7
8
9
10
